# Results in tables

In [1]:
import pandas as pd
import os
from collections import defaultdict
from IPython.display import display

In [2]:
results_folder = "results"
model_types = ["cnn14_logstft", "logmel_cnn14"]
method_names = ["saliency", "gradcam", "lime", "shap"]
exp_types_mapping = {
    "cnn14_logstft": ["clean"],
    "logmel_cnn14": ["clean", "white", "room", "horse"]
}
folder_for_tables = "csvs"

In [3]:
csv_files = []
for model in model_types:
    for method in method_names:
        for exp in exp_types_mapping[model]:
            dir_path = os.path.join(results_folder,
                                    model,
                                    f"{method}_{exp}",
                                    folder_for_tables)
            if not os.path.isdir(dir_path):
                continue
            for fname in os.listdir(dir_path):
                if fname.endswith(".csv"):
                    csv_files.append(os.path.join(dir_path, fname))

In [4]:
experiment_results = {m: {} for m in model_types}
for fpath in csv_files:
    parts = fpath.split(os.sep)
    model_type = parts[parts.index(results_folder) + 1]
    method, exp = parts[parts.index(results_folder) + 2].split("_", 1)
    exp_name = f"{method}_{exp}"
    mask_name = os.path.splitext(parts[-1])[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(['sample', 'is_correct'])
    means = df[metrics].mean()
    
    experiment_results[model_type].setdefault(exp_name, {})[mask_name] = means

In [5]:
for model, exps in experiment_results.items():
    for exp_name, mask_dict in exps.items():
        experiment_results[model][exp_name] = pd.DataFrame(mask_dict).T

In [6]:
mask_experiment_results = {m: {} for m in model_types}
for fpath in csv_files:
    parts = fpath.split(os.sep)
    model_type = parts[parts.index(results_folder) + 1]
    method, exp = parts[parts.index(results_folder) + 2].split("_", 1)
    mask_name = os.path.splitext(parts[-1])[0]
    
    df = pd.read_csv(fpath)
    metrics = df.columns.difference(['sample', 'is_correct'])
    means = df[metrics].mean()
    
    mask_experiment_results[model_type]\
      .setdefault(exp, {})\
      .setdefault(mask_name, {})[method] = means

for model, exps in mask_experiment_results.items():
    for exp_type, masks in exps.items():
        for mask_name, methods_dict in masks.items():
            mask_experiment_results[model][exp_type][mask_name] = \
                pd.DataFrame(methods_dict).T

In [7]:
column_order    = ["FF", "AI", "AG", "FidIn", "SPS", "AD", "COMP"]
invert_metrics  = {"AD", "COMP"}
diff_color_cols = {"SPS", "COMP"}

In [8]:
def highlight_extremes(col: pd.Series):
    is_invert = col.name in invert_metrics
    use_blue  = col.name in diff_color_cols
    mx, mn    = col.max(), col.min()
    styles    = []
    for v in col:
        if use_blue:
            if is_invert:
                if v == mn: styles.append('background-color: lightblue; font-weight: bold')
                elif v == mx: styles.append('background-color: lightsteelblue; font-weight: bold')
                else: styles.append('')
            else:
                if v == mx: styles.append('background-color: lightblue; font-weight: bold')
                elif v == mn: styles.append('background-color: lightsteelblue; font-weight: bold')
                else: styles.append('')
        else:
            if is_invert:
                if v == mn: styles.append('background-color: lightgreen; font-weight: bold')
                elif v == mx: styles.append('background-color: lightcoral; font-weight: bold')
                else: styles.append('')
            else:
                if v == mx: styles.append('background-color: lightgreen; font-weight: bold')
                elif v == mn: styles.append('background-color: lightcoral; font-weight: bold')
                else: styles.append('')
    return styles

In [9]:
def display_experiment_results(results):
    def _show(df, title=None):
        df = df.reindex(columns=[c for c in column_order if c in df.columns])
        styled = df.style.apply(highlight_extremes, axis=0)
        if title:
            print(f"\n--- {title} ---")
        display(styled)
    
    if isinstance(results, pd.DataFrame):
        _show(results)
        return
    
    if all(isinstance(v, pd.DataFrame) for v in results.values()):
        for name, df in results.items():
            _show(df, title=name)
        return
    
    for model, exps in results.items():
        print(f"\n===== MODEL: {model} =====")
        for name, df in exps.items():
            _show(df, title=name)

## Mel model - clean

In [10]:
display_experiment_results(experiment_results["logmel_cnn14"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.923978,19.500000,0.030776,0.652500,0.176250,34.762647,3.413026
topK_30_pos,0.924232,0.000000,0.000000,0.015000,0.700006,98.379270,9.171496
minmax_pos,0.765674,0.250000,0.049200,0.087500,0.789556,94.029495,9.025832
topK_5_pos,0.765061,0.000000,0.000000,0.022500,0.949973,97.663678,7.380256
pos_thresh_50,0.033590,0.250000,0.173707,0.015000,0.998819,97.721229,3.123119
topK_50,0.925224,0.000000,0.000000,0.020000,0.500000,98.340200,9.682342
pos_thresh_25,0.267537,0.500000,0.339988,0.022500,0.990411,97.134056,5.498956
bin,0.925166,0.000000,0.000000,0.020000,0.501420,98.346021,9.679453
minmax,0.865298,3.750000,3.182013,0.272500,0.038222,77.518508,10.372004
topK_5,0.765061,0.000000,0.000000,0.022500,0.949973,97.663678,7.380256


In [11]:
display_experiment_results(experiment_results["logmel_cnn14"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.620665,11.250000,0.937880,0.655000,0.376843,38.935879,7.311478
topK_30_pos,0.362573,7.500000,0.592328,0.447500,0.594134,60.988887,7.798657
minmax_pos,0.431521,0.750000,0.675450,0.362500,0.396169,70.568771,9.305840
topK_5_pos,0.079036,2.500000,0.000586,0.140000,0.889484,88.365241,6.940669
pos_thresh_50,0.365905,4.750000,1.725173,0.460000,0.539708,60.330152,8.580231
topK_50,0.525057,4.250000,2.337507,0.550000,0.499072,50.483148,9.684185
pos_thresh_25,0.599441,12.250000,2.856096,0.682500,0.347645,37.203954,7.528248
bin,0.719301,21.500000,2.344471,0.782500,0.195540,24.415372,4.918489
minmax,0.513954,1.250000,0.772345,0.392500,0.335306,66.666616,10.150722
topK_5,0.030442,0.000000,0.000000,0.095000,0.948799,93.796811,7.403085


In [12]:
display_experiment_results(experiment_results["logmel_cnn14"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.827965,13.750000,5.367577,0.755000,0.460687,30.268507,9.411879
topK_30_pos,0.623068,10.000000,4.776571,0.550000,0.674052,49.213681,9.252109
minmax_pos,0.755739,3.500000,1.596777,0.437500,0.547097,60.692678,9.738116
topK_5_pos,0.158620,2.000000,1.222816,0.305000,0.913671,72.956990,7.874178
pos_thresh_50,0.493181,3.000000,1.669558,0.472500,0.748397,58.130710,8.789097
topK_50,0.826212,13.750000,6.091994,0.747500,0.475981,30.978062,9.728370
pos_thresh_25,0.801759,9.500000,4.611255,0.702500,0.562260,34.795579,9.407741
bin,0.900067,29.250000,12.582630,0.932500,0.198701,8.220830,9.588828
minmax,0.788149,4.250000,2.253361,0.487500,0.392208,56.334667,10.071627
topK_5,0.158620,2.000000,1.222816,0.305000,0.913671,72.956990,7.874178


In [13]:
display_experiment_results(experiment_results["logmel_cnn14"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.927127,0.000000,0.000000,0.017500,0.500000,98.489148,9.682342
topK_30_pos,0.920322,0.000000,0.000000,0.015000,0.700006,98.376756,9.171496
minmax_pos,0.742942,0.000000,0.000000,0.017500,0.730384,98.140790,9.325663
topK_5_pos,0.644688,0.000000,0.000000,0.027500,0.949973,97.529223,7.380256
pos_thresh_50,0.059366,0.250000,0.190183,0.025000,0.996374,97.753458,4.424030
topK_50,0.927127,0.000000,0.000000,0.017500,0.500000,98.489148,9.682342
pos_thresh_25,0.514174,0.000000,0.000000,0.020000,0.955756,97.999605,7.109134
bin,0.927023,0.000000,0.000000,0.015000,0.488779,98.448436,9.704492
minmax,0.896360,0.000000,0.000000,0.080000,0.069193,93.938898,10.366691
topK_5,0.644688,0.000000,0.000000,0.027500,0.949973,97.529223,7.380256


## STFT model - clean

In [14]:
display_experiment_results(experiment_results["cnn14_logstft"]["saliency_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.804260,23.000000,0.005567,0.515000,0.246247,48.375174,5.221556
topK_30_pos,0.775904,1.000000,0.693553,0.027500,0.699991,96.591307,10.091336
minmax_pos,0.793775,1.250000,0.376756,0.045000,0.783133,94.490105,9.997836
topK_5_pos,0.616458,0.750000,0.564025,0.032500,0.949986,96.526038,8.299783
pos_thresh_50,0.031187,0.500000,0.244557,0.030000,0.999007,96.525065,3.675487
topK_50,0.791538,0.500000,0.428297,0.025000,0.499994,96.391408,10.602145
pos_thresh_25,0.220229,1.000000,0.120974,0.037500,0.991122,95.673791,6.327209
bin,0.791234,0.500000,0.428138,0.025000,0.500300,96.423009,10.601499
minmax,0.787943,10.000000,6.311180,0.345000,0.038136,66.198419,11.292009
topK_5,0.616458,0.750000,0.564025,0.032500,0.949986,96.526038,8.299783


In [15]:
display_experiment_results(experiment_results["cnn14_logstft"]["gradcam_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.724521,22.750000,0.212617,0.700000,0.224973,31.095250,4.771013
topK_30_pos,0.572838,13.500000,0.218326,0.520000,0.440970,50.516011,6.357624
minmax_pos,0.320162,0.500000,0.087145,0.100000,0.610099,90.740430,9.345481
topK_5_pos,0.191101,4.750000,0.070849,0.160000,0.835977,84.389165,7.304025
pos_thresh_50,0.267266,0.250000,0.207852,0.117500,0.762309,89.263071,8.491924
topK_50,0.585988,4.000000,1.156712,0.435000,0.499936,59.905164,10.602260
pos_thresh_25,0.398405,2.250000,0.982882,0.270000,0.600836,73.915097,9.141945
bin,0.474271,9.250000,1.076550,0.505000,0.451469,54.052505,8.473753
minmax,0.567426,1.750000,0.522353,0.262500,0.295463,75.670870,11.139101
topK_5,0.114918,0.250000,0.070103,0.042500,0.949974,95.884257,8.300031


In [16]:
display_experiment_results(experiment_results["cnn14_logstft"]["lime_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.765800,15.500000,8.450201,0.442500,0.473102,55.774762,10.191675
topK_30_pos,0.694406,10.000000,5.281612,0.345000,0.691540,66.936226,10.118867
minmax_pos,0.637870,1.000000,0.381859,0.182500,0.621597,84.515637,10.526510
topK_5_pos,0.243031,0.750000,0.135247,0.135000,0.936590,89.532042,8.526989
pos_thresh_50,0.433959,0.750000,0.290252,0.182500,0.869844,84.201880,9.094784
topK_50,0.765135,14.750000,8.862266,0.415000,0.492814,58.199032,10.616330
pos_thresh_25,0.677671,5.500000,2.461060,0.327500,0.699007,69.090076,9.976185
bin,0.788757,25.000000,13.723769,0.677500,0.272594,32.426248,10.960349
minmax,0.689437,2.750000,1.586585,0.237500,0.310267,77.675293,11.106747
topK_5,0.243031,0.750000,0.135247,0.135000,0.936590,89.532042,8.526989


In [17]:
display_experiment_results(experiment_results["cnn14_logstft"]["shap_clean"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.802138,4.500000,0.378346,0.117500,0.448744,88.001383,9.515425
topK_30_pos,0.763902,0.500000,0.387124,0.027500,0.699991,97.378701,10.091336
minmax_pos,0.455324,0.250000,0.026467,0.020000,0.733023,96.696586,10.238195
topK_5_pos,0.444992,0.250000,0.030067,0.017500,0.949986,96.537593,8.299783
pos_thresh_50,0.042822,0.750000,0.169460,0.010000,0.997583,97.548443,4.935369
topK_50,0.797199,0.500000,0.377015,0.020000,0.499994,97.530267,10.602145
pos_thresh_25,0.325784,0.000000,0.000000,0.020000,0.963614,97.009564,7.850699
bin,0.796336,0.500000,0.376223,0.020000,0.491457,97.488084,10.618963
minmax,0.758162,1.250000,0.592648,0.085000,0.072735,92.499063,11.285742
topK_5,0.444992,0.250000,0.030067,0.017500,0.949986,96.537593,8.299783


## Mask experiments clean - mel model

In [24]:
display_experiment_results(mask_experiment_results["logmel_cnn14"]["clean"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.923978,19.500000,0.030776,0.652500,0.176250,34.762647,3.413026
gradcam,0.620665,11.250000,0.937880,0.655000,0.376843,38.935879,7.311478
lime,0.827965,13.750000,5.367577,0.755000,0.460687,30.268507,9.411879
shap,0.927127,0.000000,0.000000,0.017500,0.500000,98.489148,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.924232,0.000000,0.000000,0.015000,0.700006,98.379270,9.171496
gradcam,0.362573,7.500000,0.592328,0.447500,0.594134,60.988887,7.798657
lime,0.623068,10.000000,4.776571,0.550000,0.674052,49.213681,9.252109
shap,0.920322,0.000000,0.000000,0.015000,0.700006,98.376756,9.171496



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.494280,3.500000,1.587410,0.705000,0.000435,38.660613,10.375489
gradcam,0.503141,2.000000,0.972663,0.677500,0.026292,41.577628,10.373997
lime,0.497468,2.000000,0.959788,0.677500,0.014293,42.361412,10.374873
shap,0.411536,1.750000,0.706118,0.635000,0.001600,45.858975,10.375483



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.459920,2.500000,1.172303,0.677500,0.000218,41.394705,10.375489
gradcam,0.509001,2.750000,1.287837,0.672500,0.021328,41.110658,9.803656
lime,0.496658,2.000000,0.966856,0.675000,0.013022,42.472529,10.374929
shap,0.413540,2.000000,0.717578,0.632500,0.000816,45.765689,10.375487



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.004059,0.000000,0.000000,0.032500,0.998683,97.390892,1.327483
gradcam,0.158462,0.250000,0.220969,0.220000,0.782444,83.858354,7.872584
lime,0.238357,3.250000,1.655698,0.350000,0.861851,68.144398,8.185629
shap,0.000943,0.000000,0.000000,0.027500,0.998980,97.767957,1.983555


## Mask experiments - stft model

In [25]:
display_experiment_results(mask_experiment_results["cnn14_logstft"]["clean"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.804260,23.000000,0.005567,0.515000,0.246247,48.375174,5.221556
gradcam,0.724521,22.750000,0.212617,0.700000,0.224973,31.095250,4.771013
lime,0.765800,15.500000,8.450201,0.442500,0.473102,55.774762,10.191675
shap,0.802138,4.500000,0.378346,0.117500,0.448744,88.001383,9.515425



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.775904,1.000000,0.693553,0.027500,0.699991,96.591307,10.091336
gradcam,0.572838,13.500000,0.218326,0.520000,0.440970,50.516011,6.357624
lime,0.694406,10.000000,5.281612,0.345000,0.691540,66.936226,10.118867
shap,0.763902,0.500000,0.387124,0.027500,0.699991,97.378701,10.091336



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.643463,2.500000,0.837945,0.202500,0.000142,79.326934,11.295279
gradcam,0.634945,1.750000,0.747129,0.190000,0.014195,80.017408,11.294890
lime,0.647799,2.500000,0.864931,0.217500,0.007558,78.280602,11.295109
shap,0.638557,2.000000,0.677229,0.190000,0.000569,79.908976,11.295279



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.641395,2.000000,0.750859,0.197500,0.000071,79.565120,11.295279
gradcam,0.645249,2.250000,0.778186,0.205000,0.007431,79.145813,10.504438
lime,0.648592,2.500000,0.868853,0.222500,0.006366,78.255852,11.295133
shap,0.638847,2.000000,0.679610,0.192500,0.000291,79.869429,11.295279



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.006409,0.000000,0.000000,0.012500,0.997725,97.249695,1.619247
gradcam,0.101654,0.000000,0.000000,0.050000,0.879791,95.191847,7.329040
lime,0.208866,0.250000,0.079001,0.120000,0.943331,90.890364,8.255963
shap,0.000924,0.250000,0.015379,0.012500,0.998292,97.660090,2.156133


___

## Noisy experiments - mel model

In [28]:
display_experiment_results(experiment_results["logmel_cnn14"]["saliency_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.711187,33.500000,0.008936,0.740000,0.130000,25.823075,2.517409
topK_30_pos,0.713260,0.500000,0.167238,0.022500,0.700006,97.713266,9.171496
minmax_pos,0.692836,0.500000,0.218395,0.097500,0.783585,92.282427,9.085012
topK_5_pos,0.684671,0.250000,0.000489,0.007500,0.949973,98.096582,7.380256
pos_thresh_50,0.124813,0.500000,0.231756,0.030000,0.998951,96.144690,2.895157
topK_50,0.711844,0.500000,0.152851,0.022500,0.500000,97.705562,9.682342
pos_thresh_25,0.401641,0.500000,0.373531,0.017500,0.992541,97.025918,5.222511
bin,0.711797,0.500000,0.154952,0.022500,0.502741,97.699524,9.676811
minmax,0.696040,4.750000,3.091045,0.100000,0.035331,89.111772,10.372436
topK_5,0.684671,0.250000,0.000489,0.007500,0.949973,98.096582,7.380256


In [29]:
display_experiment_results(experiment_results["logmel_cnn14"]["saliency_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.850203,27.500000,0.012627,0.702500,0.148750,29.305257,2.880497
topK_30_pos,0.850450,0.000000,0.000000,0.012500,0.700006,98.049265,9.171496
minmax_pos,0.792559,0.500000,0.203990,0.082500,0.776979,94.064579,9.124298
topK_5_pos,0.767658,0.000000,0.000000,0.017500,0.949973,97.447247,7.380256
pos_thresh_50,0.084090,0.500000,0.139204,0.017500,0.998847,97.229904,3.085727
topK_50,0.850926,0.000000,0.000000,0.012500,0.500000,98.101977,9.682342
pos_thresh_25,0.387386,0.750000,0.322935,0.025000,0.990745,96.956076,5.463568
bin,0.850942,0.000000,0.000000,0.012500,0.501831,98.092801,9.678648
minmax,0.824409,3.500000,2.288219,0.120000,0.038916,88.186978,10.371979
topK_5,0.767658,0.000000,0.000000,0.017500,0.949973,97.447247,7.380256


In [30]:
display_experiment_results(experiment_results["logmel_cnn14"]["saliency_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.790265,23.000000,0.013285,0.490000,0.256250,49.703418,4.962200
topK_30_pos,0.788512,0.000000,0.000000,0.002500,0.700006,97.261679,9.171496
minmax_pos,0.769097,0.250000,0.228000,0.067500,0.774741,92.015486,9.143423
topK_5_pos,0.694687,0.500000,0.144353,0.012500,0.949973,95.896013,7.380256
pos_thresh_50,0.107345,0.500000,0.187773,0.007500,0.998754,97.551229,3.272936
topK_50,0.789462,0.000000,0.000000,0.002500,0.500000,96.983037,9.682342
pos_thresh_25,0.384352,1.000000,0.495883,0.010000,0.989098,97.488239,5.682150
bin,0.789513,0.000000,0.000000,0.002500,0.499980,96.973540,9.682366
minmax,0.774537,10.750000,7.420244,0.275000,0.040467,72.473431,10.371812
topK_5,0.694687,0.500000,0.144353,0.012500,0.949973,95.896013,7.380256


In [31]:
display_experiment_results(experiment_results["logmel_cnn14"]["gradcam_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.614136,20.250000,0.893746,0.607500,0.279487,41.206521,5.423134
topK_30_pos,0.505541,14.750000,0.745458,0.435000,0.471790,56.855287,6.193129
minmax_pos,0.410146,1.750000,0.942405,0.162500,0.455976,86.221599,8.168055
topK_5_pos,0.216855,7.000000,0.001045,0.222500,0.785255,78.898717,6.123648
pos_thresh_50,0.367086,6.000000,1.885612,0.240000,0.578786,77.713635,7.441431
topK_50,0.552455,5.500000,2.573893,0.295000,0.499046,72.596259,9.684244
pos_thresh_25,0.468688,10.250000,2.453414,0.367500,0.425916,64.554318,6.746623
bin,0.525461,17.500000,2.581020,0.512500,0.291756,49.678478,5.421575
minmax,0.545315,1.750000,1.097447,0.205000,0.346016,81.677592,10.148300
topK_5,0.135258,0.250000,0.061111,0.072500,0.948942,94.265772,7.400324


In [32]:
display_experiment_results(experiment_results["logmel_cnn14"]["gradcam_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.668335,17.250000,1.008166,0.617500,0.328214,42.260622,6.367208
topK_30_pos,0.486541,10.500000,0.239852,0.420000,0.531212,61.049522,6.972964
minmax_pos,0.423235,0.500000,0.199748,0.227500,0.443651,82.143535,8.833017
topK_5_pos,0.146298,4.250000,0.001370,0.157500,0.846877,85.612666,6.605827
pos_thresh_50,0.359791,2.500000,0.887836,0.307500,0.578761,74.486631,8.102020
topK_50,0.595904,5.000000,2.359680,0.420000,0.499191,62.660099,9.683955
pos_thresh_25,0.527990,11.500000,3.756792,0.525000,0.397862,52.363291,7.450421
bin,0.624240,17.750000,2.535232,0.640000,0.262654,37.189112,5.514689
minmax,0.553310,1.250000,0.567462,0.272500,0.331609,76.075949,10.168407
topK_5,0.077974,0.000000,0.000000,0.060000,0.948871,95.577794,7.401692


In [33]:
display_experiment_results(experiment_results["logmel_cnn14"]["gradcam_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.668792,17.750000,0.868279,0.530000,0.343064,50.479869,6.657979
topK_30_pos,0.460496,9.500000,0.002530,0.292500,0.573245,72.849382,7.523147
minmax_pos,0.503930,0.750000,0.082838,0.147500,0.489150,88.137385,9.194560
topK_5_pos,0.120128,4.000000,0.023954,0.102500,0.882598,89.776911,6.880740
pos_thresh_50,0.449661,1.750000,0.535797,0.235000,0.606163,79.931454,8.681950
topK_50,0.612748,3.500000,1.867875,0.312500,0.499043,71.584586,9.684249
pos_thresh_25,0.575871,8.000000,1.853760,0.442500,0.437850,59.268579,8.633621
bin,0.646689,15.000000,2.474192,0.597500,0.311810,42.436289,6.832786
minmax,0.665808,2.250000,0.613855,0.255000,0.302358,78.459776,10.195316
topK_5,0.093496,0.500000,0.046791,0.037500,0.948991,96.103175,7.399400


In [34]:
display_experiment_results(experiment_results["logmel_cnn14"]["lime_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.661313,16.000000,5.634841,0.500000,0.436864,50.420390,9.111391
topK_30_pos,0.579215,10.250000,4.118605,0.360000,0.652824,64.582763,9.145879
minmax_pos,0.640139,5.500000,2.943990,0.292500,0.518476,72.855762,9.778432
topK_5_pos,0.287501,2.500000,0.941833,0.165000,0.905188,83.750481,7.976897
pos_thresh_50,0.543763,8.250000,4.268574,0.325000,0.710520,67.539896,8.939172
topK_50,0.662247,13.750000,6.180197,0.472500,0.466966,53.109267,9.745278
pos_thresh_25,0.655500,17.500000,8.751325,0.552500,0.506206,44.721665,9.431775
bin,0.698807,33.000000,9.173222,0.802500,0.187839,19.662564,8.035213
minmax,0.650653,7.500000,3.857558,0.312500,0.404169,70.803649,10.041011
topK_5,0.287501,2.500000,0.941833,0.165000,0.905188,83.750481,7.976897


In [35]:
display_experiment_results(experiment_results["logmel_cnn14"]["lime_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.798785,9.500000,3.636989,0.525000,0.464438,49.644509,9.458127
topK_30_pos,0.676898,6.500000,2.843740,0.387500,0.673583,64.811349,9.253550
minmax_pos,0.756595,2.000000,0.992075,0.265000,0.571562,78.168436,9.702441
topK_5_pos,0.266324,2.000000,1.102891,0.170000,0.917428,84.912562,7.843242
pos_thresh_50,0.545840,5.000000,2.517829,0.305000,0.778948,73.029173,8.678135
topK_50,0.799169,9.250000,4.255971,0.515000,0.477797,50.872538,9.725150
pos_thresh_25,0.765986,11.000000,4.545312,0.495000,0.598970,53.206029,9.283331
bin,0.834635,23.500000,10.195292,0.822500,0.209794,21.226862,9.733310
minmax,0.772831,2.500000,0.968496,0.272500,0.402202,75.515457,10.056722
topK_5,0.266324,2.000000,1.102891,0.170000,0.917428,84.912562,7.843242


In [36]:
display_experiment_results(experiment_results["logmel_cnn14"]["lime_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.766223,21.750000,10.250147,0.660000,0.442753,37.849872,9.047391
topK_30_pos,0.661359,15.500000,9.220467,0.412500,0.673541,59.308304,9.225868
minmax_pos,0.699449,6.750000,4.137273,0.265000,0.604778,75.024040,9.632975
topK_5_pos,0.271864,4.500000,2.608547,0.197500,0.916628,82.962081,7.847131
pos_thresh_50,0.527356,10.500000,7.369041,0.315000,0.790181,70.759820,8.655861
topK_50,0.762015,22.250000,12.026947,0.642500,0.476035,39.537862,9.728469
pos_thresh_25,0.701000,15.500000,10.393404,0.512500,0.628288,50.697775,9.277334
bin,0.780829,39.250000,20.442444,0.832500,0.289847,16.629332,9.857504
minmax,0.729969,9.500000,5.794292,0.385000,0.369308,65.290681,10.105218
topK_5,0.271864,4.500000,2.608547,0.197500,0.916628,82.962081,7.847131


In [37]:
display_experiment_results(experiment_results["logmel_cnn14"]["shap_white"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.711886,0.750000,0.244812,0.022500,0.500000,97.824995,9.682342
topK_30_pos,0.712044,0.750000,0.239618,0.022500,0.700006,97.841799,9.171496
minmax_pos,0.703274,0.250000,0.133555,0.012500,0.721409,97.693956,9.358230
topK_5_pos,0.697975,0.250000,0.142618,0.010000,0.949973,97.190341,7.380256
pos_thresh_50,0.153006,1.000000,0.414754,0.037500,0.996067,96.786615,4.462922
topK_50,0.711886,0.750000,0.244812,0.022500,0.500000,97.824995,9.682342
pos_thresh_25,0.672602,0.000000,0.000000,0.007500,0.949855,97.280215,7.272439
bin,0.711988,0.750000,0.248318,0.022500,0.488620,97.833368,9.704824
minmax,0.707670,0.000000,0.000000,0.017500,0.081259,97.210593,10.363955
topK_5,0.697975,0.250000,0.142618,0.010000,0.949973,97.190341,7.380256


In [38]:
display_experiment_results(experiment_results["logmel_cnn14"]["shap_room"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.851710,0.000000,0.000000,0.012500,0.500000,98.118956,9.682342
topK_30_pos,0.848560,0.000000,0.000000,0.012500,0.700006,98.169940,9.171496
minmax_pos,0.827405,0.000000,0.000000,0.015000,0.721625,98.417334,9.356942
topK_5_pos,0.824963,0.000000,0.000000,0.020000,0.949973,97.573138,7.380256
pos_thresh_50,0.094348,0.000000,0.000000,0.015000,0.996874,97.482746,4.248236
topK_50,0.851710,0.000000,0.000000,0.012500,0.500000,98.118956,9.682342
pos_thresh_25,0.736989,0.000000,0.000000,0.002500,0.957153,98.339164,7.097328
bin,0.851765,0.000000,0.000000,0.012500,0.486988,98.107973,9.707996
minmax,0.844405,0.000000,0.000000,0.032500,0.066837,97.179111,10.367339
topK_5,0.824963,0.000000,0.000000,0.020000,0.949973,97.573138,7.380256


In [39]:
display_experiment_results(experiment_results["logmel_cnn14"]["shap_horse"])

,FF,AI,AG,FidIn,SPS,AD,COMP
topK_50_pos,0.790700,0.250000,0.000021,0.005000,0.498750,96.701160,9.658136
topK_30_pos,0.783315,0.000000,0.000000,0.005000,0.700006,97.136170,9.171496
minmax_pos,0.753472,0.000000,0.000000,0.007500,0.727941,97.820484,9.333287
topK_5_pos,0.731783,0.500000,0.035988,0.022500,0.949973,95.778565,7.380256
pos_thresh_50,0.105709,0.500000,0.183895,0.017500,0.996972,97.933361,4.249546
topK_50,0.790346,0.000000,0.000000,0.002500,0.500000,96.830021,9.682342
pos_thresh_25,0.566565,0.250000,0.056373,0.025000,0.961684,96.388969,6.980273
bin,0.790179,0.000000,0.000000,0.005000,0.488257,96.767062,9.705524
minmax,0.788617,0.500000,0.111174,0.017500,0.067695,97.039395,10.367169
topK_5,0.731783,0.500000,0.035988,0.022500,0.949973,95.778565,7.380256


## Noisy experiments per mask - mel model

In [44]:
display_experiment_results(mask_experiment_results["logmel_cnn14"]["white"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.711187,33.500000,0.008936,0.740000,0.130000,25.823075,2.517409
gradcam,0.614136,20.250000,0.893746,0.607500,0.279487,41.206521,5.423134
lime,0.661313,16.000000,5.634841,0.500000,0.436864,50.420390,9.111391
shap,0.711886,0.750000,0.244812,0.022500,0.500000,97.824995,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.713260,0.500000,0.167238,0.022500,0.700006,97.713266,9.171496
gradcam,0.505541,14.750000,0.745458,0.435000,0.471790,56.855287,6.193129
lime,0.579215,10.250000,4.118605,0.360000,0.652824,64.582763,9.145879
shap,0.712044,0.750000,0.239618,0.022500,0.700006,97.841799,9.171496



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.551305,8.500000,4.459896,0.475000,0.000461,59.525578,10.375489
gradcam,0.514553,5.250000,1.909778,0.372500,0.021902,67.792329,10.374392
lime,0.557128,8.000000,3.849324,0.400000,0.015987,65.894878,10.374765
shap,0.479686,5.000000,1.995867,0.380000,0.001059,69.050237,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.522588,7.750000,3.455112,0.437500,0.000231,62.839069,10.375489
gradcam,0.523629,5.500000,2.505811,0.380000,0.014241,66.113705,8.922229
lime,0.555825,7.750000,3.768270,0.397500,0.014469,65.821870,10.374838
shap,0.479325,5.000000,1.981407,0.382500,0.000543,68.713834,10.375488



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.036789,0.000000,0.000000,0.040000,0.998531,96.320461,1.172918
gradcam,0.221672,0.500000,0.140789,0.100000,0.744493,91.608840,6.855313
lime,0.408775,5.250000,2.258470,0.232500,0.833223,77.525328,8.406193
shap,0.014328,0.000000,0.000000,0.035000,0.998922,96.935606,1.893806


In [45]:
display_experiment_results(mask_experiment_results["logmel_cnn14"]["room"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.850203,27.500000,0.012627,0.702500,0.148750,29.305257,2.880497
gradcam,0.668335,17.250000,1.008166,0.617500,0.328214,42.260622,6.367208
lime,0.798785,9.500000,3.636989,0.525000,0.464438,49.644509,9.458127
shap,0.851710,0.000000,0.000000,0.012500,0.500000,98.118956,9.682342



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.850450,0.000000,0.000000,0.012500,0.700006,98.049265,9.171496
gradcam,0.486541,10.500000,0.239852,0.420000,0.531212,61.049522,6.972964
lime,0.676898,6.500000,2.843740,0.387500,0.673583,64.811349,9.253550
shap,0.848560,0.000000,0.000000,0.012500,0.700006,98.169940,9.171496



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.596854,6.000000,3.486621,0.572500,0.000517,51.145875,10.375488
gradcam,0.558448,3.000000,1.595127,0.535000,0.023322,57.189180,10.374343
lime,0.598789,3.250000,1.933977,0.502500,0.015792,56.987103,10.374777
shap,0.506168,3.000000,1.742611,0.487500,0.001007,61.329066,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.560614,4.750000,2.692710,0.545000,0.000259,54.839926,10.375489
gradcam,0.565595,3.500000,1.996462,0.542500,0.016902,56.205281,9.492769
lime,0.596873,3.250000,1.912635,0.505000,0.014284,57.143269,10.374840
shap,0.507646,3.250000,1.760529,0.487500,0.000519,61.118210,10.375489



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.022937,0.000000,0.000000,0.025000,0.998625,97.008971,1.283964
gradcam,0.182925,0.000000,0.000000,0.135000,0.779125,89.180259,7.443592
lime,0.332176,2.250000,1.468688,0.197500,0.879987,81.625652,8.072819
shap,0.002504,0.000000,0.000000,0.017500,0.998952,97.195304,1.867501


In [46]:
display_experiment_results(mask_experiment_results["logmel_cnn14"]["horse"])


--- topK_50_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.790265,23.000000,0.013285,0.490000,0.256250,49.703418,4.962200
gradcam,0.668792,17.750000,0.868279,0.530000,0.343064,50.479869,6.657979
lime,0.766223,21.750000,10.250147,0.660000,0.442753,37.849872,9.047391
shap,0.790700,0.250000,0.000021,0.005000,0.498750,96.701160,9.658136



--- topK_30_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.788512,0.000000,0.000000,0.002500,0.700006,97.261679,9.171496
gradcam,0.460496,9.500000,0.002530,0.292500,0.573245,72.849382,7.523147
lime,0.661359,15.500000,9.220467,0.412500,0.673541,59.308304,9.225868
shap,0.783315,0.000000,0.000000,0.005000,0.700006,97.136170,9.171496



--- minmax_pos ---


...

,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.611034,3.500000,1.998115,0.557500,0.000539,56.133637,10.375488
gradcam,0.586634,1.000000,0.269063,0.445000,0.045041,66.064125,10.371285
lime,0.600338,4.000000,1.717499,0.497500,0.018715,61.661336,10.374649
shap,0.524776,2.000000,0.651637,0.415000,0.001090,68.324909,10.375487



--- sigmoid_pos ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.575932,3.000000,1.418633,0.512500,0.000270,61.364873,10.375489
gradcam,0.594604,2.250000,0.590318,0.470000,0.024387,63.591157,9.907195
lime,0.599717,2.750000,1.427913,0.482500,0.015105,62.475879,10.374842
shap,0.526671,2.000000,0.656936,0.422500,0.000559,68.180311,10.375488



--- pos_thresh_75 ---


,FF,AI,AG,FidIn,SPS,AD,COMP
saliency,0.021472,0.000000,0.000000,0.012500,0.998810,96.992911,1.436226
gradcam,0.252305,0.500000,0.203256,0.075000,0.799362,92.993515,7.877263
lime,0.334971,5.500000,3.827674,0.230000,0.889490,80.522536,8.001323
shap,0.003823,0.000000,0.000000,0.020000,0.998917,97.433493,1.839846
